# Malaria Classification TF 2.0

In [48]:
import numpy as np
import tensorflow 
print('Tensorflow version',tensorflow.__version__)

import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

import os
print(os.listdir("./cell_images"))

Tensorflow version 2.0.0
['Parasitized', 'Uninfected']


In [49]:
import split_folders
orig_path = './cell_images'
output_path = '../output'
split_folders.ratio(orig_path, output=output_path, seed=1, ratio=(.8, .2))

Copying files: 27560 files [01:34, 291.95 files/s]


In [50]:
data_dir = '../output'
train = data_dir+'/train/'
test = data_dir+'/val/'

In [51]:
train_image_gen = ImageDataGenerator(rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               rescale=1./255,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')

test_image_gen = ImageDataGenerator(rescale=1./255)

In [64]:
'''Set the CNN model'''
image_shape = (130,130, 3)
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=image_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
          
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

          
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [65]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 62, 62, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 29, 29, 64)       

In [66]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [67]:
batch_size = 16

train_generator = train_image_gen.flow_from_directory(train,
                                                      target_size=image_shape[:2],
                                                      color_mode='rgb',
                                                      batch_size=batch_size,
                                                      class_mode='binary')

test_generator = test_image_gen.flow_from_directory(test,
                                                    target_size=image_shape[:2],
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    class_mode='binary',
                                                    shuffle=False)

Found 22046 images belonging to 2 classes.
Found 5512 images belonging to 2 classes.


In [68]:
history = model.fit_generator(train_generator, epochs=20,
                              validation_data = test_generator,
                              callbacks=[early_stop])

Epoch 1/20
1378/1378 [==============================] - 146s 106ms/step - loss: 0.4776 - accuracy: 0.7643 - val_loss: 0.2496 - val_accuracy: 0.9443
Epoch 2/20
1378/1378 [==============================] - 160s 116ms/step - loss: 0.2110 - accuracy: 0.9324 - val_loss: 0.1677 - val_accuracy: 0.9436
Epoch 3/20
1378/1378 [==============================] - 152s 110ms/step - loss: 0.1913 - accuracy: 0.9418 - val_loss: 0.1418 - val_accuracy: 0.9557
Epoch 4/20
1378/1378 [==============================] - 154s 112ms/step - loss: 0.1844 - accuracy: 0.9431 - val_loss: 0.1438 - val_accuracy: 0.9539
Epoch 5/20
1378/1378 [==============================] - 158s 115ms/step - loss: 0.1761 - accuracy: 0.9450 - val_loss: 0.1331 - val_accuracy: 0.9574
Epoch 6/20
1378/1378 [==============================] - 155s 113ms/step - loss: 0.1786 - accuracy: 0.9447 - val_loss: 0.1368 - val_accuracy: 0.9563
Epoch 7/20
1378/1378 [==============================] - 146s 106ms/step - loss: 0.1759 - accuracy: 0.9447 - val_

In [70]:
model.save('./malaria_model')

INFO:tensorflow:Assets written to: ./malaria_model\assets
